In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

In [5]:
# Reflect Tables into SQLAlchemy ORM
#Python SQL toolkit and ORM
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [6]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [8]:
# Reflect an existing databse into a new model
Base = automap_base()
# Reflect the tables
Base.prepare(engine, reflect=True)

In [9]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [10]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [11]:
# Create our session from Python to the database in order to query the database
session = Session(engine)

Exploratory Climate Analysis

In [17]:
# Display the Measurement row's columns and data in dictionary format
first_row = session.query(Measurement).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1c54b47c3c8>,
 'tobs': 65.0,
 'prcp': 0.08,
 'station': 'USC00519397',
 'date': '2010-01-01',
 'id': 1}

In [13]:
# Display the Station row's columns and data in dictionary format
first_station_row = session.query(Station).first()
first_station_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1c54b47ccf8>,
 'name': 'WAIKIKI 717.2, HI US',
 'id': 1,
 'elevation': 3.0,
 'latitude': 21.2716,
 'station': 'USC00519397',
 'longitude': -157.8168}

In [24]:
# Get the latest date from Measurement
latest_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()

In [30]:
#Get the date 12 months ago from latest date
year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print(year_ago)

2016-08-23


In [40]:
# Design a query to retrieve the last 12 months of precipatation data and plot results
#for row in session.query(Measurement.date, Measurement.prcp).all():
    #print(row)
prcp_year = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date > year_ago).\
    order_by(Measurement.date).all()

#for year in prcp_year:
#    print("Date: {0}, Precip: {1}".format(year.date, year.prcp))


In [42]:
# Load query results into DataFrame and set index to the date
m_df = pd.DataFrame(prcp_year, columns=['Date', 'Precip'])
m_df.set_index('Date', inplace=True)
m_df

,Precip
Date,
2016-08-24,0.08
2016-08-24,2.15
2016-08-24,2.28
2016-08-24,NaN
2016-08-24,1.22
2016-08-24,2.15
2016-08-24,1.45
2016-08-25,0.08
2016-08-25,0.08
